In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load cleaned data exported from clean_eda.ipynb)
df = pd.read_csv("/content/cleaned_mental_health_data.csv")  # Update with your actual output file

df.head()

,Timestamp,Gender,Country,Occupation,self_employed,family_history,treatment,Days_Indoors,Growing_Stress,Changes_Habits,Mental_Health_History,Mood_Swings,Coping_Struggles,Work_Interest,Social_Weakness,mental_health_interview,care_options,Days_Indoors_Clean,Year,Weekday
0,2014-08-27 11:29:00,Female,United States,Corporate,No,No,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Maybe,NaN,2014,Wednesday
1,2014-08-27 11:31:00,Female,United States,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,No,NaN,2014,Wednesday
2,2014-08-27 11:32:00,Female,United States,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Yes,NaN,2014,Wednesday
3,2014-08-27 11:37:00,Female,United States,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,Maybe,Yes,NaN,2014,Wednesday
4,2014-08-27 11:43:00,Female,United States,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Yes,NaN,2014,Wednesday


In [16]:
# --- Feature Engineering ---

# Example: Group countries into broader regions (simple mapping)
region_map = {
    'United States': 'North America',
    'Canada': 'North America',
    'India': 'Asia',
    'United Kingdom': 'Europe',
    # Add more as needed
}
df['Region'] = df['Country'].map(region_map).fillna('Other')

# Example: Composite feature for mental stress indicators
stress_cols = ['Growing_Stress', 'Mood_Swings']
df['Stress_Flag'] = df[stress_cols].apply(lambda row: row.eq('Yes').sum(), axis=1)

# Drop original stress columns if desired
# df.drop(columns=stress_cols, inplace=True)

In [17]:
# --- Encoding ---

# Label encode binary and ordinal fields
label_cols = ['treatment', 'family_history', 'Region']
label_encoders = {}
for col in label_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Create age groups if 'Age' or equivalent exists
#df['Age_Group'] = pd.cut(df['Age'], bins=[0, 18, 30, 45, 60, 100],
                         #labels=['Teen', 'YoungAdult', 'Adult', 'MiddleAge', 'Senior'])

# One-hot encode multi-class categoricals
df = pd.get_dummies(df, columns=['Gender', 'Weekday'], drop_first=True)


In [18]:
# --- Scaling ---

scaler = StandardScaler()
numeric_cols = ['Days_Indoors_Clean']
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# --- Split dataset ---
X = df.drop(columns=['treatment'])
y = df['treatment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Save split sets if needed
# X_train.to_csv("X_train.csv", index=False)
# y_train.to_csv("y_train.csv", index=False)

print("\nPreprocessing complete. Ready for modeling.")


Preprocessing complete. Ready for modeling.


/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [19]:
print(df[numeric_cols].describe())
print(df[numeric_cols].isnull().sum())

       Days_Indoors_Clean
count                 0.0
mean                  NaN
std                   NaN
min                   NaN
25%                   NaN
50%                   NaN
75%                   NaN
max                   NaN
Days_Indoors_Clean    248510
dtype: int64


In [20]:
#  Train-Test Split
#  Since the data is fully preprocessed, the next step is to split it into training and testing sets for modeling.

from sklearn.model_selection import train_test_split

# Define target and features
X = df.drop(columns=['treatment'])  # features
y = df['treatment']                # target

# Perform the split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Train set: {X_train.shape}, Test set: {X_test.shape}")

Train set: (198808, 26), Test set: (49702, 26)
